In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install transformers
!pip install torchaudio

Mounted at /content/gdrive
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pydub

In [ ]:
import pandas as pd

# Carregar o dataset
dataset_path = '/content/gdrive/My Drive/Colab Notebooks/audios/dataset.csv'
data = pd.read_csv(dataset_path)

# Corrigindo os caminhos na coluna 'Location'
data['Location'] = data['Location'].str.replace('/content/drive', '/content/gdrive', regex=False)

# Verificar se o replace foi bem sucedido
print(data['Location'].head())

# Salvar o dataframe corrigido, se necessário
data.to_csv(dataset_path, index=False)

0    /content/gdrive/My Drive/Colab Notebooks/audio...
1    /content/gdrive/My Drive/Colab Notebooks/audio...
2    /content/gdrive/My Drive/Colab Notebooks/audio...
3    /content/gdrive/My Drive/Colab Notebooks/audio...
4    /content/gdrive/My Drive/Colab Notebooks/audio...
Name: Location, dtype: object


In [ ]:
from pydub import AudioSegment
import numpy as np
import torch

def load_and_preprocess_audio(audio_path):
    # Carrega o áudio com pydub, que suporta diversos formatos
    audio = AudioSegment.from_file(audio_path)
    # Converte para um array de onda de áudio do tipo numpy
    samples = np.array(audio.get_array_of_samples())
    # Se o áudio for estéreo, converte para mono
    if audio.channels == 2:
        samples = samples.reshape((-1, 2)).mean(axis=1)
    waveform = torch.tensor(samples).float() / 2**15  # Conversão para tensor e normalização
    waveform = waveform.unsqueeze(0)  # Adiciona uma dimensão de canal
    sample_rate = audio.frame_rate  # Obter a taxa de amostragem
    return waveform, sample_rate

# Exemplo de carregamento de um arquivo de áudio
waveform, sample_rate = load_and_preprocess_audio(data['Location'][0])

Aqui começa o fine tune

### **1. Preparando DataLoaders**

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

# Divisão do dataset em treino e validação
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

class AudioDataset(Dataset):
    def __init__(self, dataframe, label_mapping):
        self.dataframe = dataframe
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        audio_path = self.dataframe.iloc[idx]['Location']
        sentiment = self.dataframe.iloc[idx]['Sentiment']
        label = self.label_mapping[sentiment]
        waveform, sample_rate = load_and_preprocess_audio(audio_path)
        return waveform.squeeze(0), label

    @staticmethod
    def collate_fn(batch):
        waveforms, labels = zip(*batch)
        waveforms_padded = pad_sequence(waveforms, batch_first=True, padding_value=0.0)
        labels_tensor = torch.tensor(labels, dtype=torch.long)
        return waveforms_padded, labels_tensor

# Mapeamento de sentimentos para inteiros
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}

# Instâncias do dataset
train_dataset = AudioDataset(train_df, label_mapping)
val_dataset = AudioDataset(val_df, label_mapping)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=AudioDataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=AudioDataset.collate_fn)

### 2. **Definição do Modelo e Função de Perda** [OPÇÃO 1]

> Add blockquote



In [ ]:
## OFICIAL

# #esse funciona

# from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Config
# import torch.nn as nn

# # Carregando a configuração com número apropriado de labels
# config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base-960h", num_labels=3)

# # Carregando o modelo com configuração ajustada para classificação
# model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-base-960h", config=config)
# model.to('cuda')  # Transferindo modelo para GPU

# # Você pode continuar usando CrossEntropyLoss com este modelo
# criterion = nn.CrossEntropyLoss().to('cuda')

In [ ]:
# def train_model(model, train_loader, criterion, optimizer, epochs):
#     model.train()
#     for epoch in range(epochs):
#         total_loss = 0
#         for waveforms, labels in train_loader:
#             waveforms = waveforms.to('cuda')
#             labels = labels.to('cuda')

#             optimizer.zero_grad()

#             outputs = model(waveforms).logits  # Saída do modelo
#             outputs = torch.nn.functional.log_softmax(outputs, dim=-1)

#             # Calcular a perda
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item()

#         print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_loader)}")

### 2. **Definição do Modelo e Função de Perda** [OPÇÃO 1]


In [ ]:
#teste
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Config
import torch.nn as nn

config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-large-xlsr-53-portuguese", num_labels=3)
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-large-xlsr-53-portuguese", config=config)

model.to('cuda')  # Transferindo modelo para GPU

# # Você pode continuar usando CrossEntropyLoss com este modelo
criterion = nn.CrossEntropyLoss().to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53-portuguese and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for waveforms, labels in train_loader:
            waveforms = waveforms.to('cuda')
            labels = labels.to('cuda')

            optimizer.zero_grad()

            outputs = model(waveforms).logits  # Ajuste aqui
            loss = criterion(outputs, labels)  # Sem softmax/log_softmax
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_loader)}")

## Rodando o fine-tune ##

In [ ]:
import torch.optim as optim

# Configuração do otimizador Adam
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

train_model(model, train_loader, criterion, optimizer, epochs)

Epoch 1: Loss = 1.1169581974382365
Epoch 2: Loss = 1.106603939672966
Epoch 3: Loss = 1.1250473007614359
Epoch 4: Loss = 1.1327493637719195
Epoch 5: Loss = 1.101901147173438
Epoch 6: Loss = 1.096873396918887
Epoch 7: Loss = 1.0981731993374808
Epoch 8: Loss = 1.099005447857546
Epoch 9: Loss = 1.108064667427496
Epoch 10: Loss = 1.1034198328033908


### 3.**Validação**

In [ ]:
def evaluate_model(model, val_loader, criterion, device='cuda'):
    model.eval()  # Coloca o modelo em modo de avaliação
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Desativa a computação de gradientes
        for waveforms, labels in val_loader:
            waveforms = waveforms.to(device)
            labels = labels.to(device)

            outputs = model(waveforms).logits
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted_labels = torch.max(outputs, dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)

    avg_loss = total_loss / len(val_loader)
    accuracy = correct_predictions / total_predictions
    return avg_loss, accuracy

# Uso da função
val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

## Primeiro modelo: com crossentropy + facebook/wav2vec2-base-960h
# Validation Loss: 1.1009
# Validation Accuracy: 0.3188

#se não funcionar, adicionar uma cama extra no final (testar c/ Keras)

Validation Loss: 1.0930
Validation Accuracy: 0.3188


## Resultados:

**Primeiro modelo:** com crossentropy + facebook/wav2vec2-base-960h + 5 epochs
Validation Loss: 1.1009
Validation Accuracy: 0.3188

**Segundo modelo:** com crossentropy + facebook/wav2vec2-xlsx-portuguese
Validation Loss: 1.0991
Validation Accuracy: 0.3188

In [ ]:
def predict_sentiment(model, audio_path, device='cuda'):
    waveform, sample_rate = load_and_preprocess_audio(audio_path)
    model.eval()
    waveform = waveform.to(device)
    with torch.no_grad():
        outputs = model(waveform).logits
        probabilities = torch.nn.functional.softmax(outputs, dim=-1)
        predictions = torch.argmax(probabilities, dim=-1)
        print("Raw predictions over time:", predictions)
        predicted_label_index, _ = torch.mode(predictions)
        print("Most frequent predicted index:", predicted_label_index)
    return predicted_label_index.item()

In [ ]:
label_sentiment =  {0:'positive', 1: 'negative', 2:'neutral'}

In [ ]:
# Supondo que o modelo e o label_mapping estejam carregados
audio_path = "/content/gdrive/My Drive/Colab Notebooks/audios/test/carol-chorando.ogg"
predicted_label_index = predict_sentiment(model, audio_path)
predicted_label = label_sentiment.get(predicted_label_index, "Label not found")
predicted_label

Raw predictions over time: tensor([2], device='cuda:0')
Most frequent predicted index: tensor(2, device='cuda:0')


'neutral'